# OpenAI Assistants - Building Agentic RAG with the Function Calling, Retrieval, and Code Interpreter Tools

Today we'll explore using OpenAI's Python SDK to create, manage, and use the OpenAI Assistant API!

We'll be doing the following in today's notebook:

1. Task 1: Simple Assistant
2. Task 2: Adding Tools
  - Task 2a: Creating an Assistant with File Search Tool
  - Task 2b: Creating an Assistant with Code Interpreter Tool
  - Task 2c: Creating an Assistant with Function Calling Tool



---

Colab Specific Instructions:

To get started, please make a copy of this notebook using `File > Save a copy in Drive`

![image](https://i.imgur.com/rNzMEfs.png)

You will be expected to submit a GitHub link to the completed notebook, so if you're completing the assignment on Colab - you'll want to download the notebook when you have completed it.

## Dependencies

We'll start, as we usually do, with some dependiencies and our API key!

In [1]:
from rich import print

%load_ext rich

In [2]:
!pip install -qU openai

In [3]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

assert os.getenv("OPENAI_API_KEY") is not None


## Task 1: Simple Assistant

Let's create a simple Assistant to understand more about how the API works to start!

### OpenAI Client

At the core of the OpenAI Python SDK is the Client!

> NOTE: For ease of use, we'll start with the synchronous `OpenAI()`. OpenAI does provide an `AsyncOpenAI()` that you could leverage as well!

In [6]:
from openai import OpenAI

client = OpenAI()

### Creating An Assistant

Leveraging what we know about the OpenAI API from previous sessions - we're going to start by simply initializing an Assistant.

Before we begin, we need to think about a few customization options we have:

- `name` - Straight forward enough, this is what our Assistant's name will be
- `instructions` - similar to a system message, but applied at an Assistant level, this is how we can guide the Assistant's tone, behaviour, functionality, and more!
- `model` - this will allow us to choose which model we would prefer to use for our Assistant

Let's start by setting some instructions for our Assistant.



In [42]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown Fill out the fields below to add your Assistant's name, instructions, and desired model!

name = "General Chatbot"  # @param {type: "string"}
instructions = "Answer in a professional manner."  # @param {type: "string"}
model = "gpt-4o"  # @param ["gpt-3.5-turbo", "gpt-4-turbo-preview", "gpt-4", "gpt-4o"]

### Initialize Assistant

Now that we have our desired name, instruction, and model - we can initialize our Assistant!

In [43]:
assistant = client.beta.assistants.create(
    name=name,
    instructions=instructions,
    model=model,
)

Let's examine our `assistant` object and see what we find!

In [44]:
assistant


Assistant(
    id='asst_aSPvLTXMt0Z1bfUAVZcNTcO6',
    created_at=1718401905,
    description=None,
    instructions='Answer in a professional manner.',
    metadata={},
    model='gpt-4o',
    name='General Chatbot',
    object='assistant',
    tools=[],
    response_format='auto',
    temperature=1.0,
    tool_resources=ToolResources(code_interpreter=None, file_search=None),
    top_p=1.0
)

There are a number of useful parameters here, but we'll call out a few:

- `id` - since we may have multiple Assistant's, knowing which Assistant we're interacting with will help us ensure the desired user experience!
- `description` - A natrual language description of our Assistant could help others understand what it's supposed to do!
- `file_ids` - if we wanted to use the Retrieval tool, this would let us know what files we had given our Assistant

### Creating a Thread

Behind the scenes our Assistant is powered by the idea of "threads".

You can think of threads as individual conversations that interact with the Assistant.

Let's create a thread now!

In [45]:
thread = client.beta.threads.create()

Let's look at our `thread` object.

In [46]:
thread


Thread(
    id='thread_Yjm4riVN4Hb04PYXZuqwqHv7',
    created_at=1718401907,
    metadata={},
    object='thread',
    tool_resources=ToolResources(code_interpreter=None, file_search=None)
)

Notice some key attributes:

- `id` - since each Thread is like a conversation, we need some way to specify which thread we're dealing with when interacting with them
- `tool_resources` - this will become more relevant as we add tools since we'll need a way to verify which tools we have access to when interacting with our Assistant

### Adding Messages to Our Thread

Now that we have our Thread (or conversation) we can start adding messages to it!

Let's add a simple message that asks about how our Assistant is feeling.

Notice the parameters we're leveraging:

- `thread_id` - since each Thread is like a conversation, we need some way to address a specific conversation. We can use `thread.id` to do this.
- `role` - similar to when we used our chat completions endpoint, this parameter specifies who the message is coming from. You can leverage this in the same ways you would through the chat completions endpoint.
- `content` - this is where we can place the actual text our Assistant will interact with

> NOTE: Feel free to substitute a relevant message based on the Assistant you created

In [47]:
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content=f"How old are you?"
)

Again, let's examine our `message` object!

In [48]:
message


Message(
    id='msg_OwONDWmC74Pq1op5uB8eqoZJ',
    assistant_id=None,
    attachments=[],
    completed_at=None,
    content=[TextContentBlock(text=Text(annotations=[], value='How old are you?'), type='text')],
    created_at=1718401909,
    incomplete_at=None,
    incomplete_details=None,
    metadata={},
    object='thread.message',
    role='user',
    run_id=None,
    status=None,
    thread_id='thread_Yjm4riVN4Hb04PYXZuqwqHv7'
)

### Running Our Thread

Now that we have an Assistant, and we've given that Assistant a Thread, and we've added a Message to that Thread - we're ready to run our Assistant!

Notice that this process lets us add (potentially) multiple messages to our Assistant. We can leverage that behaviour for few/many-shot examples, and more!

In [49]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown We can also override the Assistant's instructions when we run a thread.

# @markdown Use one of the [Prompt Principles for Instruction](https://arxiv.org/pdf/2312.16171v1.pdf) to improve the likeliehood of a correct or valuable response from your Assistant.

additional_instructions = """Your task is to return the description of an algorithm that can answer the user's question, explaining like they are 5 years old. 
Come up with a creative example. If no such algorithm can be made, say so. Do not generate code. Only generate 1 example."""  # @param {type: "string"}

Let's run our Thread!

In [50]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions=additional_instructions
)

Now that we've run our thread, let's look at the object!

In [51]:
run


Run(
    id='run_R1qbwGhFYtPo03w1hJOlIL1N',
    assistant_id='asst_aSPvLTXMt0Z1bfUAVZcNTcO6',
    cancelled_at=None,
    completed_at=None,
    created_at=1718401917,
    expires_at=1718402517,
    failed_at=None,
    incomplete_details=None,
    instructions="Your task is to return the description of an algorithm that can answer the user's question, explaining like they are 5 years old. \nCome up with a creative example. If no such algorithm can be made, say so. Do not generate code. Only generate 1 example.",
    last_error=None,
    max_completion_tokens=None,
    max_prompt_tokens=None,
    metadata={},
    model='gpt-4o',
    object='thread.run',
    parallel_tool_calls=True,
    required_action=None,
    response_format='auto',
    started_at=None,
    status='queued',
    thread_id='thread_Yjm4riVN4Hb04PYXZuqwqHv7',
    tool_choice='auto',
    tools=[],
    truncation_strategy=TruncationStrategy(type='auto', last_messages=None),
    usage=None,
    temperature=1.0,
    top_p=1.

Notice we have access to a few very powerful parameters in this `run` object.

- `completed_at` - this will help us determine when we can expect to retrieve a response
- `failed_at` - this can highlight any issues our run ran into
- `status` - is another way we can understand how the flow is going

### Retrieving Our Run

Now that we've created our run, let's retrieve it.

We're going to wrap this in a simple loop to make sure we're not retrieving it too early.

In [52]:
import time

while run.status == "in_progress" or run.status == "queued":
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

In [53]:
print(run.status)

completed

Now that our run is completed - we can retieve the messages from our thread!

Notice that our run helps us understand how things are going - but it isn't where we're going to find our responses or messages. Those are added on the backend into our thread.

This leads to a simple, but important, flow:

1. We add messages to a thread.
2. We create a run on that thread.
3. We wait until the run is finished.
4. We check our thread for the new messages.

### Checking Our Thread

Now we can get a list of messages from our thread!

In [54]:
messages = client.beta.threads.messages.list(thread_id=thread.id)

In [55]:
messages.data[0]


Message(
    id='msg_AKbjeemGk7xFaFQuw2PwRqOK',
    assistant_id='asst_aSPvLTXMt0Z1bfUAVZcNTcO6',
    attachments=[],
    completed_at=None,
    content=[
        TextContentBlock(
            text=Text(
                annotations=[],
                value="Imagine we have a magical age-guessing robot named Timmy. Timmy is very smart and good at numbers! But Timmy can't tell you his age directly because he's a robot and doesn't age like humans do.\n\nInstead, let's play a little game. Think of how old you are, and then imagine that Timmy's age is just like a very wise, grown-up helper in a toy shop. Timmy has been helping kids for as long as you can imagine, maybe even forever in a toy land where nobody keeps track of years. So, Timmy's age is a mystery, just like magic!\n\nNow, whenever you ask Timmy how old he is, he wants you to think of him as someone who is always there to help, no matter the number of years. So, while Timmy might not give you an exact age, you can think of him 

### Streaming Our Runs

With recent upgrades to the Assistant API - we can now *stream* our outputs!

In order to do this - we'll need something called an `EventHandler` which will help us to decide on what actions to take based on the output of the LLM.

Let's build it below!

In [56]:
from typing_extensions import override
from openai import AssistantEventHandler
from sys import stdout

stdout.flush()

class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        stdout.write(f"\nassistant > ")

    @override
    def on_text_delta(self, delta, snapshot):
        stdout.write(delta.value)

Now we can create our `run` and stream the output as it comes in!

In [57]:
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions=additional_instructions,
    event_handler=EventHandler(),
) as stream:
    stream.until_done()


assistant > Imagine we have a magical garden with a special tree named Treebot. Treebot is very, very old and wise, and it knows everything about taking care of the garden. But, just like with magic trees, we don't really count how old Treebot is in the same way we count birthdays for kids.

Instead, let's say Treebot's age is like the number of beautiful flowers it has helped grow in the garden. Suppose every year it helps a new batch of flowers bloom beautifully. If you see many flowers, then you know Treebot has been there for a long time!

So, when you ask Treebot how old it is, imagine it has been there forever, always helping each flower grow happily. The exact number of years doesn't matter because Treebot's job is to make the garden wonderful all the time. That way, you can always think of Treebot as your forever-friend who loves making the garden pretty for everyone.

## Task 2: Adding Tools

Now that we have an understanding of how Assistant works, we can start thinking about adding tools.

We'll go through 3 separate tools and explore how we can leverage them!

Let's start with the most familiar tool - the Retriever!


### Task 2a: Creating an Assistant with the File Search Tool

The first thing we'll want to do is create an assistant with the File Search tool.

This is also going to require some data. We'll provided data - but you're very much encouraged to use your own files to explore how the Assistant works for your use case.

#### Collect and Add Data to Vector Store

1. First, we need some data.
2. Second, we need to add the data to our Assistant!

Let's start with grabbing some data!

In [61]:
!curl -O https://raw.githubusercontent.com/dbredvick/paul-graham-to-kindle/main/paul_graham_essays.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 3003k    0 23999    0     0  80432      0  0:00:38 --:--:--  0:00:38 81907
100 3003k  100 3003k    0     0  5857k      0 --:--:-- --:--:-- --:--:-- 5924k


Now we can upload our file to our Vector Store!

Pay attention to [this](https://platform.openai.com/docs/assistants/tools/file-search/supported-files) documentation to see what kinds of files can be uploaded.

> NOTE: Per the OpenAI [docs](https://platform.openai.com/docs/assistants/tools/file-search/vector-stores) The maximum file size is 512 MB and no more than 2,000,000 tokens (computed automatically when you attach a file)

In [62]:
vector_store = client.beta.vector_stores.create(name="Paul Graham Essay Compilation")

In [63]:
file_paths = ["paul_graham_essays.txt"]
file_streams = [open(path, "rb") for path in file_paths]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

Let's look at what our `file_batch` contains!

In [64]:
while file_batch.status != "completed":
  time(1)

#### Create and Use Assistant

Now that we have our file - we can attach it to an Assistant, and we can give that Assistant the ability to use it for retrieval through the Retrieval tool!

> NOTE: Your first GB is free and beyond that, usage is billed at $0.10/GB/day of vector storage. There are no other costs associated with vector store operations.

In [65]:
fs_assistant = client.beta.assistants.create(
    name=name,
    instructions=instructions,
    model=model,
    tools=[{"type": "file_search"}],
)

In [66]:
fs_assistant = client.beta.assistants.update(
    assistant_id=fs_assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [67]:
fs_assistant


Assistant(
    id='asst_hJ4ZpUpPWAfHAfovKoeZxknz',
    created_at=1718402090,
    description=None,
    instructions='Answer in a professional manner.',
    metadata={},
    model='gpt-4o',
    name='General Chatbot',
    object='assistant',
    tools=[FileSearchTool(type='file_search', file_search=None)],
    response_format='auto',
    temperature=1.0,
    tool_resources=ToolResources(
        code_interpreter=None,
        file_search=ToolResourcesFileSearch(vector_store_ids=['vs_ERIwlgjBbjCdMHUonq5I2PSc'])
    ),
    top_p=1.0
)

In [72]:
fs_thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "What did Paul Graham say about effective altruism?",
        }
    ]
)

We can use an extension of the `EventHandler` that we created above to stream our `run`!

Let's add a few things:

1. A `on_tool_call_created` function which tells us which tool is being used.
2. A `on_message_done` that includes citations that were used by our File Search tool - this is like return the context *and* the response that we saw in our Pythonic RAG implementation.

In [75]:
stdout.flush()


class FSEventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        stdout.write(f"\nassistant > ")

    @override
    def on_tool_call_created(self, tool_call):
        stdout.write(f"\nassistant > {tool_call.type}\n")

    @override
    def on_message_done(self, message) -> None:
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        stdout.write(message_content.value + "\n")
        stdout.write("\n".join(citations) + "\n")


Let's look at the final result!

In [76]:
with client.beta.threads.runs.stream(
    thread_id=fs_thread.id,
    assistant_id=fs_assistant.id,
    event_handler=FSEventHandler(),
) as stream:
    stream.until_done()


assistant > file_search

assistant > Paul Graham has expressed a skeptical view of effective altruism. He criticizes it on several points: he feels that it "makes a parody of something important" and that it "causes people to wear themselves out doing things that don't matter." He does not conclude definitively whether effective altruism is net good or bad, but he leans towards the idea that it might be more likely bad[0][1].
[0] paul_graham_essays.txt
[1] paul_graham_essays.txt


### Task 2b: Creating an Assistant with the Code Interpreter Tool

Now that we've explored the Retrieval Tool - let's try the Code Interpreter tool!

The process will be almost exactly the same - but we can explore a different query, and we'll add our file at the Message level!

In [77]:
ci_assistant = client.beta.assistants.create(
    name=name + "+ Code Interpreter",
    instructions=instructions,
    model=model,
    tools=[{"type": "code_interpreter"}],
)

In [78]:
!git clone https://github.com/ali-ce/datasets.git

Cloning into 'datasets'...


In [79]:
file = client.files.create(
    file=open("datasets/Y-Combinator/Startups.csv", "rb"), purpose="assistants"
)

In the following example, we'll also see how we can package the Thread creation with the Message adding step!

> NOTE: Files added at the message/thread level will not be available to the Assistant outside of that Thread.

In [81]:
ci_thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "What kind of file is this?",
            "attachments": [
                {"file_id": file.id, "tools": [{"type": "code_interpreter"}]}
            ],
        }
    ]
)

We'll once again need to create an `EventHandler`, except this time it will have an `on_tool_call_delta` method which will let us see the output of the code interpreter tool as well!

> NOTE: Remember that we create runs at the *thread* level - and so don't need the message object to continue.

In [83]:
stdout.flush()


class CIEventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        stdout.write(f"\nassistant > ")

    @override
    def on_text_delta(self, delta, snapshot):
        stdout.write(delta.value)

    def on_tool_call_created(self, tool_call):
        stdout.write(f"\nassistant > {tool_call.type}\n")

    def on_tool_call_delta(self, delta, snapshot):
        if delta.type == "code_interpreter":
            if delta.code_interpreter.input:
                stdout.write(delta.code_interpreter.input)
            if delta.code_interpreter.outputs:
                stdout.write(f"\n\noutput >")
                for output in delta.code_interpreter.outputs:
                    if output.type == "logs":
                        stdout.write(f"\n{output.logs}")


Once again, we can use the streaming interface thanks to creating our `EventHandler`!

In [84]:
with client.beta.threads.runs.stream(
    thread_id=ci_thread.id,
    assistant_id=ci_assistant.id,
    instructions=additional_instructions,
    event_handler=CIEventHandler(),
) as stream:
    stream.until_done()


assistant > code_interpreter
import mimetypes

# File path
file_path = '/mnt/data/file-Sa9K7FjxfuUaXFUXBeJg7LSv'

# Determine the file type
file_type, _ = mimetypes.guess_type(file_path)
file_type
assistant > I can't tell what the file is just by its name. Let me take a closer look to see if I can figure it out for you. One second!# Let's try to read the first few bytes of the file to identify its type
with open(file_path, 'rb') as file:
    file_header = file.read(20)
file_header
assistant > Alright, imagine you have a magical book. When you open the first page, you see that it's talking about different companies, their status, and the year. So, from the first few words, it looks like the file is some kind of list or table about companies. This type of file is often called a CSV file, which stands for Comma-Separated Values. It's like a big table with lots of information organized neatly into rows and columns.

And there you go!

We've fit our Assistant with an awesome Code Interpreter that lets our Assistant run code on our provided files!

### Task 2c: Creating an Assistant with a Function Calling Tool

Let's finally create an Assistant that utilizes the Function Calling API.

We'll start by creating a function that we wish to be called.

We'll utilize DuckDuckGo search to allow our Assistant to have the most up to date information!

In [85]:
!pip install -qU duckduckgo_search

In [91]:
from duckduckgo_search import DDGS


def duckduckgo_search(query):
    with DDGS() as ddgs:
        results = [r for r in ddgs.text(query, max_results=5)]
        return "\n".join(result["body"] for result in results)

Let's test our function to make sure it behaves as we expect it to.

In [99]:
duckduckgo_search("Who is the CEO of Apple?")

"Chief Executive Officer. Tim Cook is the CEO of Apple and serves on its board of directors. Before being named CEO in August 2011, Tim was Apple's chief operating officer and was responsible for all of the company's worldwide sales and operations, including end-to-end management of Apple's supply chain, sales activities, and service and ...\nApple Leadership - Apple Executive Profiles is a webpage that introduces the top executives of Apple, the world's most innovative company. You can learn about their backgrounds, roles, and achievements, as well as their vision and values for Apple. Whether you are a fan, a customer, or a potential partner, you will find this webpage informative and inspiring.\nWebsite. Apple Leadership Profile. Signature. Timothy Donald Cook (born November 1, 1960) [1] is an American business executive who is the current chief executive officer of Apple Inc. Cook had previously been the company's chief operating officer under its co-founder Steve Jobs. [2] Cook jo

Now we need to express how our function works in a way that is compatible with the OpenAI Function Calling API.

We'll want to provide a `JSON` object that includes what parameters we have, how to call them, and a short natural language description.

In [100]:
ddg_function = {
    "name": "duckduckgo_search",
    "description": "Answer non-technical questions. ",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type:": "string",
                "description": "The search query to use. For example: 'Who is the current Goalie of the Colorado Avalance?'",
            }
        },
        "required": ["query"],
    },
}

#### ❓ Question

Why does the description key-value pair matter?

#### ❓ Answer

The `description` key-value pair is important because it helps the Assistant understand what the function does and how it can be used. You can think of this as being similar to a function's docstring that helps developers understand how to use the function.

Now when we create our Assistant - we'll want to include the function description as a tool using the following format.

In [142]:
fc_assistant = client.beta.assistants.create(
    name=name + " + Function Calling",
    instructions=instructions,
    tools=[{"type": "function", "function": ddg_function}],
    model=model,
)

Now we can create our thread, and attach our message to it - just as we've been doing!

In [166]:
fc_thread = client.beta.threads.create()
fc_message = client.beta.threads.messages.create(
    thread_id=fc_thread.id,
    role="user",
    content="Can you describe the Twitter beef between Elon and Sam Altman?",
)

Once again, we'll need an `EventHandler` for the streaming response - notice that this time we're utilizing a few new methods:

1. `handle_requires_action` - this will handle whatever action needs to take place in *our local environment*.
2. `submit_tool_outputs` - this will let us submit the resultant outputs from our local function call back to the Assistant run in the required format.

To be very clear and explicit - we'll be following this pattern:

1. Make a call to the LLM which will decide if a local function call is required.
2. If a local function call is required - send a response that indicates a local function call is required.
3. Call the function using the arguments provided by the LLM.
4. Return the response from the function call with LLM provided arguements.
5. Receive a response based on the output of the functional call from the LLM.

In [167]:
class FCEventHandler(AssistantEventHandler):
    @override
    def on_event(self, event):
        # Retrieve events that are denoted with 'requires_action'
        # since these will have our tool_calls
        if event.event == "thread.run.requires_action":
            run_id = event.data.id  # Retrieve the run ID from the event data
            self.handle_requires_action(event.data, run_id)

    def handle_requires_action(self, data, run_id):
        tool_outputs = []

        for tool in data.required_action.submit_tool_outputs.tool_calls:
            print(tool.function.arguments)
            if tool.function.name == "duckduckgo_search":
                tool_outputs.append(
                    {
                        "tool_call_id": tool.id,
                        "output": duckduckgo_search(tool.function.arguments),
                    }
                )

        # Submit all tool_outputs at the same time
        self.submit_tool_outputs(tool_outputs, run_id)

    def submit_tool_outputs(self, tool_outputs, run_id):
        # Use the submit_tool_outputs_stream helper
        with client.beta.threads.runs.submit_tool_outputs_stream(
            thread_id=self.current_run.thread_id,
            run_id=self.current_run.id,
            tool_outputs=tool_outputs,
            event_handler=EventHandler(),
        ) as stream:
            stream.until_done()

Thanks to our event handler - we can stream this process in our notebook!

In [168]:
stdout.flush()
with client.beta.threads.runs.stream(
    thread_id=fc_thread.id, assistant_id=fc_assistant.id, event_handler=FCEventHandler()
) as stream:
    stream.until_done()


{"query": "Elon Musk and Sam Altman Twitter beef"}

{"query": "Elon Musk Sam Altman conflict details"}


assistant > It appears that there might not be widely known or documented recent "Twitter beef" or conflict between Elon Musk and Sam Altman. Both are prominent figures in the tech industry, Musk as the CEO of companies like Tesla and SpaceX, and Altman as the CEO of OpenAI. They have been known to have their differences, especially in perspectives on artificial intelligence and its implications.

Elon Musk has been a vocal critic of the potential dangers of AI, advocating for strict regulation to ensure it is developed safely. Sam Altman, on the other hand, as the head of OpenAI, promotes the development of AI but also recognizes the need for safety and ethical guidelines. Their differing views could potentially lead to public disagreements.

However, without specific recent information about a direct conflict or "Twitter beef" between the two, it's important to consider that their public interactions might be more rooted in professional debates rather than personal animosity. If you